In [54]:
import random

# benchmark function settings
xmin = -2
xmax = -xmin

ymin = -2
ymax = -ymin

def rosenbrock(x, y):
    a = 0
    b = 1
    return (a-x)**2 + b*(y-x**2)**2


# test
import plotly.graph_objects as go
import numpy as np

def plot_benckmark_function(func_name):

    # plot x, y axis value settings
    xs = np.arange(xmin, xmax, 0.25)
    ys = np.arange(ymin, ymax, 0.25)
    X, Y = np.meshgrid(xs, ys)

    if func_name == 'rosenbrock':
        Z = rosenbrock(X, Y)
    elif func_name == 'rastrigin':
        Z = rastrigin(X, Y)
    else:
        raise Exception('Unknown function; Please select one between (rosenbrock | rastrigin)')

    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
    fig.update_layout(title = func_name.capitalize(),
                    width=500, 
                    height=500,
                    margin=dict(l=65, r=50, b=65, t=90))
    
    fig.show()

In [55]:
plot_benckmark_function('rosenbrock')

In [54]:
def pso(n=20):


    # 0. Initial settings
    n_iterations = 20

    # 1. Generate initial swarm
        # n = number of particles
    xs = [random.uniform(xmin, xmax) for _ in range (n)] # generating random x coordinates
    ys = [random.uniform(ymin, ymax) for _ in range (n)] # generating random y coordinates
    particles = list(zip(xs, ys)) # creating list of (x, y) tuples
    pis = [(1, 1)] * n # generating initial 
    pg = (2, 2)

    # 2. Repeat, for each particle, update xi, pi, pg, vi
    for _ in (n_iterations):
        for i, particle in enumerate(particles):

            # Evaluate function at position
            z_particle = rosenbrock(particle[0], particle[1]) # particle[0] = x coord; particle[1] = y coord
            pi_particle = pis[i]
            if z_particle < pi_particle: 
                pis[i] = particle

            # Update pg
            z_pg = rosenbrock(pg[0], pg[1])
            pg = min(z_particle, z_pg)

            # Update velocity
            vi = 

            # Update position
            xi = 



In [37]:
xs = np.arange(xmin, xmax, 0.25)
ys = np.arange(ymin, ymax, 0.25)
X, Y = np.meshgrid(xs, ys)

Z = rosenbrock(X, Y)

# swarm
xs = [random.uniform(xmin, xmax) for _ in range (n)] # generating random x coordinates
ys = [random.uniform(ymin, ymax) for _ in range (n)] # generating random y coordinates
positions = list(zip(xs, ys)) # creating list of (x, y) tuples


zs = [rosenbrock(x, y) for x, y in positions]

fig = go.Figure(data=[
                      go.Surface(z=Z, 
                                 x=X, 
                                 y=Y, 
                                 opacity=0.7), 
                      
                        go.Scatter3d(x=[], 
                                    y=[], 
                                    z=[],
                                    mode = 'markers',
                                    marker={
                                            'size':5,
                                            'color':'black',
                                            'opacity':0.8
                                    }
                        )
                      ])

xss = [np.array(xs) + i for i in range(5)]
yss = [np.array(ys) for i in range(5)]
# zss = [np.array(zs) + i for i in range(5)]

positionss = list(zip(xss, yss)) # creating list of (x, y) tuples

zss = [rosenbrock(x, y) for x, y in positionss]

frames = [go.Frame(data= [
                    #   go.Surface(z=Z, 
                    #              x=X, 
                    #              y=Y, 
                    #              opacity=0.7),
                                                     
                          go.Scatter3d(
                                       x=xss[k], 
                                       y=yss[k],
                                       z=zss[k])],
                   
                   traces= [0],
                   name=f'frame{k}'      
                  ) for k in range(5)]

fig.update(frames=frames)

fig.update_layout(updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))      ])])])

fig.show();